# HealTAC 2023: MiADE (Medical information AI Data Extractor) Tutorial Demo

Install MiADE with deploy key

In [1]:
! pip install gdown
import gdown


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
!mkdir -p ~/.ssh
gdown.download('https://drive.google.com/uc?id=1Grou2v7t3nyziiOAYtIoL1O0xsiTh_lv', '/root/.ssh/id_rsa')
!chmod 600 ~/.ssh/id_rsa
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

**You may need to restart the runtime aafter this step as prompted if running in Colab**

In [ ]:
!pip install git+ssh://git@github.com/uclh-criu/miade.git

Download data & models

In [ ]:
!pip install https://huggingface.co/kormilitzin/en_core_med7_lg/resolve/main/en_core_med7_lg-any-py3-none-any.whl

In [5]:
MODEL_DIR = './models/'
DATA_DIR = './data/'

!mkdir $MODEL_DIR
!mkdir $DATA_DIR

In [6]:
import gdown
gdown.download('https://drive.google.com/uc?id=11NZNd7k4J5t-wofDWsPxHMJ3Jg5QmbiI', MODEL_DIR + "miade_meds_model_a146c741501cf1f7.zip")
gdown.download('https://drive.google.com/uc?id=17s999FIotRenltR6gr_f8ZjdaXc-u1Gx', MODEL_DIR + 'miade_problems_model_f25ec9423958e8d6.zip')

Downloading...
From (uriginal): https://drive.google.com/uc?id=11NZNd7k4J5t-wofDWsPxHMJ3Jg5QmbiI
From (redirected): https://drive.google.com/uc?id=11NZNd7k4J5t-wofDWsPxHMJ3Jg5QmbiI&confirm=t&uuid=610a24e3-075c-482d-995f-ef1f224250a0
To: /home/jennifer/Documents/miade-tutorials/notebooks/models/miade_meds_model.zip
100%|████████████████████████████████████████| 309M/309M [00:15<00:00, 20.5MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=17s999FIotRenltR6gr_f8ZjdaXc-u1Gx
From (redirected): https://drive.google.com/uc?id=17s999FIotRenltR6gr_f8ZjdaXc-u1Gx&confirm=t&uuid=27b7340b-245d-4441-adab-9cb55012768b
To: /home/jennifer/Documents/miade-tutorials/notebooks/models/miade_problems_model.zip
100%|████████████████████████████████████████| 468M/468M [00:29<00:00, 15.7MB/s]


'./models/miade_problems_model.zip'

In [7]:
!wget https://raw.githubusercontent.com/uclh-criu/miade-tutorials/master/notebooks/models/config.yaml -P $MODEL_DIR
!wget https://raw.githubusercontent.com/uclh-criu/miade-tutorials/master/notebooks/data/meds_allergies_cdb.csv -P $DATA_DIR
!wget https://raw.githubusercontent.com/uclh-criu/miade-tutorials/master/notebooks/data/wikipedia_sample.txt -P $DATA_DIR

--2023-06-13 17:09:19--  https://raw.githubusercontent.com/uclh-criu/miade-tutorials/master/notebooks/models/config.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155 [text/plain]
Saving to: ‘MODEL_DIR/config.yaml’

config.yaml         100%[===================>]     155  --.-KB/s    in 0s      

2023-06-13 17:09:19 (6.64 MB/s) - ‘MODEL_DIR/config.yaml’ saved [155/155]

--2023-06-13 17:09:20--  https://raw.githubusercontent.com/uclh-criu/miade-tutorials/master/notebooks/data/meds_allergies_cdb.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8000::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTT

In [8]:
from pathlib import Path
from typing import List, Dict, Optional

from miade.core import NoteProcessor
from miade.note import Note
from miade.annotators import Annotator
from miade.concept import Concept, Category
from miade.dosageextractor import DosageExtractor
from miade.utils.miade_cat import MiADE_CAT

## Part 1: Extracting concepts and dosages from notes using the full MiADE pipeline

### Configuring the MiADE processor

```NoteProcessor``` is the MiADE core. It is initialised with a model directory path that contains all the MedCAT model pack .zip files we would like to use in our pipeline, and a `config.yaml` file that maps an alias to the model IDs (model IDs can be found in MedCAT `model_cards` or usually will be in the name) and annotators we would like to use:

```
models:
  problems: f25ec9423958e8d6
  meds/allergies: a146c741501cf1f7
annotators:
  problems: ProblemsAnnotator
  meds/allergies: MedsAllergiesAnnotator
 ```

In [10]:
miade = NoteProcessor(Path(MODEL_DIR))

[2023-06-13 17:12:32,971] [INFO] miade.core: Found config file models/config.yaml
[2023-06-13 17:12:32,973] [INFO] miade.core: Loading MedCAT models from models
/home/jennifer/Documents/miade/venv/lib/python3.8/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_md' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[2023-06-13 17:12:41,550] [INFO] miade.dosageextractor: Loaded drug dosage extractor with model en_core_med7_lg


Once ```NoteProcessor``` is initialised, we can add annotators by the aliases we have specified in ```config.yaml``` to our processor. 

When adding annotators, we have the option to add [NegSpacy](https://spacy.io/universe/project/negspacy) to the MedCAT spaCy pipeline, which implements the [NegEx algorithm](https://www.sciencedirect.com/science/article/pii/S1532046401910299) (Chapman et al. 2001) for negation detection. This allows the models to perform simple rule-based negation detection in the absence of MetaCAT models.

In [11]:
miade.add_annotator("problems", use_negex=True)

[2023-06-13 17:12:47,071] [INFO] miade.core: Added ProblemsAnnotator to processor
[2023-06-13 17:12:47,079] [INFO] miade.core: Added Negex context detection for ProblemsAnnotator


In [12]:
miade.add_annotator("meds/allergies")

[2023-06-13 17:12:48,427] [INFO] miade.core: Added MedsAllergiesAnnotator to processor


In [13]:
# print the MedCAT model cards to check that it's been mapped to the correct annotators
miade.print_model_cards()

ProblemsAnnotator: {
  "Model ID": "f25ec9423958e8d6",
  "Last Modified On": "12 June 2023",
  "History (from least to most recent)": [
    "0b130195e6964e66",
    "353abfe8f57e2009",
    "af37e525e37cdb1a"
  ],
  "Description": "MiADE problems development model unsupervised trained on MIMIC-III. cdb: full_condition_list_MedCAT_cdb.csv; vocab: medcat model 811218c0b819c304 (feb 2022) | Packaged with MetaCAT model(s) presence, relevance",
  "Source Ontology": "SNOMEDUK_May2022 MiADE subset",
  "Location": "gae03:gae/miade/models/problems/, miade-dev:vol/models/problems/",
  "MetaCAT models": [
    {
      "Category Name": "presence",
      "Description": "Blank meta model with tokenizer trained on MIMIC III",
      "Classes": {
        "confirmed": 0,
        "negated": 1,
        "suspected": 2
      },
      "Model": "lstm"
    },
    {
      "Category Name": "relevance",
      "Description": "Blank meta model with tokenizer trained on MIMIC III",
      "Classes": {
        "historic"

### Creating a Note

Next we will create a ```Note``` object which contains the text we would like to extract concepts and dosages from

In [14]:
text = """
Suspected heart failure

PMH:
prev history of Hypothyroidism
MI 10 years ago


Current meds:
Losartan 100mg daily
Atorvastatin 20mg daily
Paracetamol 500mg tablets 2 tabs qds prn

Allergies:
Penicillin - rash

Referred with swollen ankles and shortness of breath since 2 weeks.
"""

In [15]:
note = Note(text)

### Extracting concepts and dosages

MiADE currently extracts concepts in SNOMED CT. Each concept has:
- `name`: name of concept
- `id`: concept ID
- `category`: type of concept e.g. problems, medictions
- `start`: start index of concept span
- `end`: end index of concept span
- `dosage`: for medication concepts
- `negex`: Negex result if configured
- `meta`: Meta annotations if MetaCAT models are used

The dosages associated with medication concepts are extracted by the built-in MiADE `DosageExtractor`, using a combination of NER model [Med7](https://github.com/kormilitzin/med7) and [the CALIBER rule-based drug dose lookup algorithm](https://rdrr.io/rforge/CALIBERdrugdose/). It returns:

- `dose`
- `duration`
- `frequency`
- `route`

The output format is directly translatable to HL7 CDA but can also easily be converted to FHIR.

In [16]:
concepts = miade.get_concept_dicts(note)

In [17]:
concepts

[{'name': 'hypothyroidism (historic)',
  'id': '161443002',
  'category': 'PROBLEM',
  'start': 46,
  'end': 60,
  'dosage': None,
  'negex': False,
  'meta': [{'name': 'relevance',
    'value': 'HISTORIC',
    'confidence': 0.999841570854187},
   {'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.999984622001648}],
  'debug': None},
 {'name': 'dyspnea',
  'id': '267036007',
  'category': 'PROBLEM',
  'start': 243,
  'end': 262,
  'dosage': None,
  'negex': False,
  'meta': [{'name': 'relevance',
    'value': 'PRESENT',
    'confidence': 0.9999895095825195},
   {'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.9906362891197205}],
  'debug': None},
 {'name': 'swollen ankle',
  'id': '267039000',
  'category': 'PROBLEM',
  'start': 224,
  'end': 238,
  'dosage': None,
  'negex': False,
  'meta': [{'name': 'relevance',
    'value': 'PRESENT',
    'confidence': 0.9999951124191284},
   {'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.985640

#### Handling existing records - deduplication

MiADE is built to handle existing medication records from EHR systems that can be sent alongside the note. It will perform basic deduplication matching on id for existing record concepts.

In [18]:
# create list of concepts that already exists in patient record
record_concepts = [
    Concept(id="40930008", name="hypothyroidism", category=Category.PROBLEM),
    Concept(id="267039000", name="swollen ankle", category=Category.PROBLEM)
]

In [19]:
miade.get_concept_dicts(note=note, record_concepts=record_concepts)

[{'name': 'hypothyroidism (historic)',
  'id': '161443002',
  'category': 'PROBLEM',
  'start': 46,
  'end': 60,
  'dosage': None,
  'negex': False,
  'meta': [{'name': 'relevance',
    'value': 'HISTORIC',
    'confidence': 0.999841570854187},
   {'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.999984622001648}],
  'debug': None},
 {'name': 'dyspnea',
  'id': '267036007',
  'category': 'PROBLEM',
  'start': 243,
  'end': 262,
  'dosage': None,
  'negex': False,
  'meta': [{'name': 'relevance',
    'value': 'PRESENT',
    'confidence': 0.9999895095825195},
   {'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.9906362891197205}],
  'debug': None},
 {'name': 'eruption of skin',
  'id': '271807003',
  'category': 'PROBLEM',
  'start': 204,
  'end': 208,
  'dosage': None,
  'negex': False,
  'meta': [{'name': 'relevance',
    'value': 'PRESENT',
    'confidence': 0.9999878406524658},
   {'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.999

#### Removing annotators

In [20]:
# we can also remove annotators if no longer wanted
miade.remove_annotator("meds/allergies")

[2023-06-13 17:14:23,768] [INFO] miade.core: Removed MedsAllergiesAnnotator from processor


In [21]:
miade.get_concept_dicts(note=note, record_concepts=record_concepts)

[{'name': 'hypothyroidism (historic)',
  'id': '161443002',
  'category': 'PROBLEM',
  'start': 46,
  'end': 60,
  'dosage': None,
  'negex': False,
  'meta': [{'name': 'relevance',
    'value': 'HISTORIC',
    'confidence': 0.999841570854187},
   {'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.999984622001648}],
  'debug': None},
 {'name': 'dyspnea',
  'id': '267036007',
  'category': 'PROBLEM',
  'start': 243,
  'end': 262,
  'dosage': None,
  'negex': False,
  'meta': [{'name': 'relevance',
    'value': 'PRESENT',
    'confidence': 0.9999895095825195},
   {'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.9906362891197205}],
  'debug': None},
 {'name': 'eruption of skin',
  'id': '271807003',
  'category': 'PROBLEM',
  'start': 204,
  'end': 208,
  'dosage': None,
  'negex': False,
  'meta': [{'name': 'relevance',
    'value': 'PRESENT',
    'confidence': 0.9999878406524658},
   {'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.999

## Part 2: Customising MiADE

### 2.1 Building custom MedCAT models

This part of the tutorial will go through the basics of building a MedCAT model that can be used with MiADE.

For more information on **MedCAT**:

- [Github](https://github.com/CogStack/MedCAT)
- [MedCAT Tutorials](https://github.com/CogStack/MedCATtutorials)
- [Paper](https://arxiv.org/abs/2010.01165)

In [22]:
from miade.model_builders.cdbbuilder import CDBBuilder
from miade.model_builders.vocabbuilder import VocabBuilder

In [23]:
CDB_DATA_PATH = DATA_DIR + "meds_allergies_cdb.csv"

In [24]:
TEXT_DATA_PATH = DATA_DIR + "wikipedia_sample.txt"

#### **Create a CDB (concept databse) builder**

The MiADE `CDBBuilder` uses SNOMED CT and builds from a preprocessed CSV in the following example format:

| cui      | name                  | ontologies | name_status |
| -------- | --------------------- | ---------- | ----------- |
| 59927004 | hepatic failure       | SNO        | P           |
| 59927004 | hepatic insufficiency | SNO        | A           |


Where "P" denotes primary name and "A" denotes alternate name. 

See MedCAT Tutorials For more information on building CDB and Vocabs with custom ontologies

In [25]:
cdbbuilder = CDBBuilder(
    temp_dir=(Path.cwd() / Path(".temp")),  # temp dir for saving data
    custom_data_paths=[Path(CDB_DATA_PATH)]
)

In [26]:
# process and create the CDB object
cdbbuilder.preprocess()
cdb = cdbbuilder.create_cdb()

In [27]:
# inspect the vocab
cdb.vocab

{'acetaminophen': 1,
 '500mg': 3,
 'oral': 4,
 'tablet': 4,
 'paracetamol': 1,
 'ibuprofen': 1,
 '200mg': 1,
 'penicillin': 1,
 'coconut': 1,
 'metformin': 1,
 'hydrochloride': 1,
 'peanut': 1,
 'butter': 1,
 'a': 2,
 'rash': 2,
 'complaining': 1,
 'of': 3,
 'facial': 1,
 'swelling': 1,
 'symptoms': 2,
 'depression': 2,
 'symptom': 2,
 'depressive': 2}

#### **Create a Vocab builder**

The `Vocab` takes a free text as input and trains a corpus of word embeddings on it using [Word2Vec](https://www.tensorflow.org/tutorials/text/word2vec). 

Using `VocabBuilder` we can pass a list of texts we would like to create our vocab with - here we are using some wikipedia sample text.

In [28]:
with open(TEXT_DATA_PATH, "r", encoding="utf-8") as training_data:
    training_data_list = [line.strip() for line in training_data]

In [29]:
# inspect the data
training_data_list

['Lung cancer, also known as bronchial carcinoma, since about 98–99% of all lung cancers are carcinomas, is a malignant lung tumor characterized by uncontrolled cell growth in tissues of the lung. Lung carcinomas derive from transformed, malignant cells that originate as epithelial cells, or from tissues composed of epithelial cells. Other lung cancers, such as the rare sarcomas of the lung, are generated by the malignant transformation of connective tissues (i.e. nerve, fat, muscle, bone), which arise from mesenchymal cells. Lymphomas and melanomas (from lymphoid and melanocyte cell lineages) can also rarely result in lung cancer.',
 'In time, this uncontrolled growth can spread beyond the lung – either by direct extension, by entering the lymphatic circulation, or via the hematogenous, bloodborne spread – the process called metastasis – into nearby tissue or other, more distant parts of the body. Most cancers that start in the lung, known as primary lung cancers, are carcinomas. The 

In [30]:
# create a builder
vocab_builder = VocabBuilder()

In [31]:
# create the new vocab
vocab = vocab_builder.create_new_vocab(
    training_data_list=training_data_list,
    cdb=cdb,
    config=cdb.config
)

In [32]:
# inspect the vocab
vocab.vocab.keys()

dict_keys(['lung', 'cancer', 'also', 'known', 'as', 'bronchial', 'carcinoma', 'since', 'about', '98', '99', 'of', 'all', 'cancers', 'are', 'carcinomas', 'is', 'a', 'malignant', 'tumor', 'characterized', 'by', 'uncontrolled', 'cell', 'growth', 'in', 'tissues', 'the', 'derive', 'from', 'transformed', 'cells', 'that', 'originate', 'epithelial', 'or', 'composed', 'other', 'such', 'rare', 'sarcomas', 'generated', 'transformation', 'connective', 'i', 'e', 'nerve', 'fat', 'muscle', 'bone', 'which', 'arise', 'mesenchymal', 'lymphomas', 'and', 'melanomas', 'lymphoid', 'melanocyte', 'lineages', 'can', 'rarely', 'result', 'time', 'this', 'spread', 'beyond', 'either', 'direct', 'extension', 'entering', 'lymphatic', 'circulation', 'via', 'hematogenous', 'bloodborne', 'process', 'called', 'metastasis', 'into', 'nearby', 'tissue', 'more', 'distant', 'parts', 'body', 'most', 'start', 'primary', 'two', 'main', 'types', 'small', 'sclc', 'non', 'nsclc', 'common', 'symptoms', 'coughing', 'including', 'up'

#### **Create a model pack**

In [33]:
# saves the model pack into .zip with the vocab and cdb we just created - this is what we load into miade
vocab_builder.make_model_pack(cdb=cdb, save_name="miade_example")

#### Run training from script (try in own time)

MiADE provides scripts for automatically building MedCAT model packs, unsupervised training, supervised training steps, and the creation and training of MetaCAT models. 

The ```--synthetic-data-path``` option allows you to add synthetically generated training data in CSV format to the supervised and MetaCAT training steps. The CSV should have the following format:

| text                          | cui               | name                       | start | end | relevance | presence  | laterality |
| ----------------------------- | ----------------- | -------------------------- | ----- | --- | --------- | --------- | -------------------- |
| no history of liver failure | 59927004 | hepatic failure      | 14     | 26  | historic  | negated | none                 

In [ ]:
# Path to MedCAT model pack to use for training
MODEL_PACK_PATH = ""

# Path to MedCATTrainer JSON export (annotations)
MEDCAT_JSON_EXPORT = ""

# Path to synthetic data in CSV format
SYNTHETIC_CSV_PATH = ""

# Path to the tokenizer to use for MetaCAT model training
TOKENIZER_PATH = ""

# List of category names for MetaCAT e.g. presence, relevance - will create MetaCAT folders in the format "meta_$CATEGORY_NAME"
CATEGORY_NAMES = []

# Path to the MetaCAT model folders to train
METACAT_MODL_PATH = ""

In [ ]:
# Trains unsupervised training step of MedCAT model
!miade train $MODEL_PACK_PATH $TEXT_DATA_PATH --tag "miade-example"

In [ ]:
# Trains supervised training step of MedCAT model
!miade train-supervised $MODEL_PACK_PATH $MEDCAT_JSON_EXPORT --synthetic-data-path $SYNTHETIC_CSV_PATH

In [ ]:
# Creates BBPE tokenizer for MetaCAT
!miade create-bbpe-tokenizer $TEXT_DATA_PATH

In [ ]:
# Initialises MetaCAT models to do training on
!miade create-metacats $TOKENIZER_PATH $CATEGORY_NAMES

In [ ]:
# Trains the MetaCAT Bi-LSTM models
!miade train-metacats $METACAT_MODEL_PATH $MEDCAT_JSON_EXPORT --synthetic-data-path $SYNTHETIC_CSV_PATH

In [ ]:
# Packages MetaCAT models with the main MedCAT model pack
!miade add_metacat_models $MODEL_PACK_PATH $METACAT_MODEL_PATH

#### **And more...**

[**MiADE training dashboard**](https://github.com/uclh-criu/miade/tree/master/streamlit_app): A streamlit app that allows you to interactively inspect synthetic data and train MetaCAT models

### 2.2 Creating custom MiADE annotators

We can add custom annotators with more specialised postprocessing steps to MiADE by subclassing ```Annotator``` and initialising ```NoteProcessor``` with the list of custom annotators 

```Annotator``` methods include:

- ```.get_concepts()```: returns MedCAT output as MiADE ```Concepts```
- ```.add_dosages_to_concepts()```: uses the MiADE built-in ```DosageExtractor``` to add dosages associated with medication concepts
- ```.deduplicate()```: filters duplicate concepts in list 

In [34]:
# subclass Annotator
class CustomAnnotator(Annotator):
    def __init__(self, cat: MiADE_CAT):
        super().__init__(cat)
        # we need to include MEDICATIONS in concept types so MiADE processor will also extract dosages
        self.concept_types = [Category.MEDICATION, Category.ALLERGY]
    
    def postprocess(self, concepts: List[Concept]) -> List[Concept]:
        # some example post-processing code
        reactions = ["271807003"]
        allergens = ["764146007"]
        for concept in concepts:
            if concept.id in reactions:
                concept.category = Category.REACTION
            elif concept.id in allergens:
                concept.category = Category.ALLERGY
        return concepts
    
    def __call__(
        self,
        note: Note,
        record_concepts: Optional[List[Concept]] = None,
        dosage_extractor: Optional[DosageExtractor] = None,
    ):
        concepts = self.get_concepts(note)
        concepts = self.postprocess(concepts)
        # run dosage extractor if given
        if dosage_extractor is not None:
            concepts = self.add_dosages_to_concepts(dosage_extractor, concepts, note)
        concepts = self.deduplicate(concepts, record_concepts)

        return concepts
        

Don't forget to add custom annotator to `config.yaml`
```
models:
  problems: f25ec9423958e8d6
  meds/allergies: a146c741501cf1f7
  custom: a146c741501cf1f7
annotators:
  problems: ProblemsAnnotator
  meds/allergies: MedsAllergiesAnnotator
  custom: CustomAnnotator
 ```

In [36]:
# pass custom annotator to miade
miade = NoteProcessor(Path(MODEL_DIR), custom_annotators=[CustomAnnotator])

[2023-06-13 17:23:52,343] [INFO] miade.core: Found config file models/config.yaml
[2023-06-13 17:23:52,344] [INFO] miade.core: Loading MedCAT models from models
/home/jennifer/Documents/miade/venv/lib/python3.8/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_md' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[2023-06-13 17:24:03,532] [INFO] miade.dosageextractor: Loaded drug dosage extractor with model en_core_med7_lg


In [37]:
miade.add_annotator("problems", use_negex=True)

[2023-06-13 17:24:15,242] [INFO] miade.core: Added ProblemsAnnotator to processor


In [38]:
miade.add_annotator("custom")

[2023-06-13 17:24:16,157] [INFO] miade.core: Added CustomAnnotator to processor


In [39]:
miade.get_concept_dicts(note)

[{'name': 'hypothyroidism (historic)',
  'id': '161443002',
  'category': 'PROBLEM',
  'start': 46,
  'end': 60,
  'dosage': None,
  'negex': False,
  'meta': [{'name': 'relevance',
    'value': 'HISTORIC',
    'confidence': 0.999841570854187},
   {'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.999984622001648}],
  'debug': None},
 {'name': 'dyspnea',
  'id': '267036007',
  'category': 'PROBLEM',
  'start': 243,
  'end': 262,
  'dosage': None,
  'negex': False,
  'meta': [{'name': 'relevance',
    'value': 'PRESENT',
    'confidence': 0.9999895095825195},
   {'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.9906362891197205}],
  'debug': None},
 {'name': 'swollen ankle',
  'id': '267039000',
  'category': 'PROBLEM',
  'start': 224,
  'end': 238,
  'dosage': None,
  'negex': False,
  'meta': [{'name': 'relevance',
    'value': 'PRESENT',
    'confidence': 0.9999951124191284},
   {'name': 'presence',
    'value': 'CONFIRMED',
    'confidence': 0.985640

In [40]:
# or we can use .process() to get the Concept objects directly
concepts = miade.process(note)
for concept in concepts:
    if concept.category == Category.ALLERGY or concept.category == Category.REACTION:
        print(concept)

{name: breaking out - eruption, id: 271807003, category: Category.REACTION, start: 204, end: 208, dosage: None, negex: False, meta: None} 
{name: penicillin, id: 764146007, category: Category.ALLERGY, start: 191, end: 201, dosage: None, negex: False, meta: None} 


## End of tutorial
